In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import Document
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

/Users/ashwin/Applications/Workflow/notetakker/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = ChatOllama(model = "deepseek-r1:7b", callbacks=callback_manager)

In [3]:
plan_template = """ 
I need you to help me break down the following long-form writing instruction into multiple subtasks. Each subtask will guide the writing of one paragraph in the essay, and should include the main points and word count requirements for that paragraph.

The writing instruction is as follows:

{intructions}

Please break it down in the following format, with each subtask taking up one line:

Paragraph 1 - Main Point: [Describe the main point of the paragraph, in detail] - Word Count: [Word count requirement, e.g., 400 words]

Paragraph 2 - Main Point: [Describe the main point of the paragraph, in detail] - Word Count: [word count requirement, e.g. 1000 words].

...

Make sure that each subtask is clear and specific, and that all subtasks cover the entire content of the writing instruction. Do not split the subtasks too finely; each subtask's paragraph should be no less than 200 words and no more than 1000 words. Do not output any other content. As this is an ongoing work, omit open-ended conclusions or other rhetorical hooks.

"""

In [4]:
write_template = """ 
You are an excellent writing assistant. I will give you an original writing instruction and my planned writing steps. I will also provide you with the text I have already written. Please help me continue writing the next paragraph based on the writing instruction, writing steps, and the already written text.

Writing instruction:

{intructions}

Writing steps:

{plan}

Already written text:

{text}

Please integrate the original writing instruction, writing steps, and the already written text, and now continue writing {STEP}. If needed, you can add a small subtitle at the beginning. Remember to only output the paragraph you write, without repeating the already written text.
"""

In [5]:
plan_prompt = ChatPromptTemplate([
        ('user', plan_template)
    ])

In [6]:
plan_chain = plan_prompt | llm | StrOutputParser()

In [7]:
if __name__ == "__main__":
    test_instruction = "Give me an indepth plan on how to write tests with pytest. I want to learn how to do TDD."
    
    result = plan_chain.invoke({"intructions": test_instruction})
    
    # Print the result
    print("Generated Writing Plan:")
    print(result)

<think>
Okay, so I'm trying to figure out how to write a test with pytest using TDD. I've heard that TDD stands for Test-Driven Development, which sounds cool but I'm not exactly sure how it all works together with pytest. Let me start by breaking down what each part means.

First off, TDD is about writing tests before you write the actual code. The idea is to prove that your code works as expected by creating unit tests and integration tests first. That makes sense because if I can show that my code does what it's supposed to do, then when I actually write the code later, it will meet those requirements.

Now, pytest is a Python testing framework that makes running and writing tests easier than some other options like unittest or nose. I know pytest uses markings which help in describing test cases. So maybe one of the subtasks should cover how to structure a project so that pytest can find all my test files easily. That sounds important because if the project is organized well, pytes

In [8]:
write_prompt = ChatPromptTemplate([
    ('user', write_template)
    ])

write_chain = write_prompt | llm | StrOutputParser()

In [9]:
if __name__ == "__main__":
    test_instruction = "Write a 1500-word essay about the impact of artificial intelligence on modern society, covering its benefits, potential risks, and ethical considerations."
    test_plan = "Paragraph 1 - Main Point: Introduction to AI and its growing influence - Word Count: 200 words"
    test_text = "Artificial Intelligence (AI) has become an integral part of our modern society, revolutionizing various aspects of our daily lives and industries."
    
    result = write_chain.invoke({
        "intructions": test_instruction,
        "plan": test_plan,
        "text": test_text,
        "STEP": "Paragraph 1"
    })
    
    print("Generated Paragraph:")
    print(result)

<think>
Okay, so I need to help expand on the introduction of an essay about AI's impact on modern society. The user has provided some existing text and wants me to continue with Paragraph 1, which should cover the main points: introducing AI, its growing influence, benefits, potential risks, ethical considerations, and a conclusion.

First, looking at the already written text: "Artificial Intelligence (AI) has become an integral part of our modern society, revolutionizing various aspects of our daily lives and industries." That's good. I should build on this by explaining how AI is integrated into different areas, maybe providing examples like healthcare or transportation to illustrate its impact.

I also need to mention the benefits it brings, such as increased efficiency in logistics, better medical diagnoses, and personalized education systems. These points show positive impacts that are tangible for readers.

Next, addressing potential risks: data privacy concerns because AI often

In [10]:
def write_markdown_file(content, filename):
    with open(f"{filename}.md", "w") as f:
        f.write(content)

In [11]:
def planning_node(state):
    """take the initial prompt and write a plan to make a long doc"""
    print("---PLANNING THE WRITING---")
    initial_prompt = state['initial_prompt']
    num_steps = int(state['num_steps'])
    num_steps += 1

    plan = plan_chain.invoke({"intructions": initial_prompt})

    return {"plan": plan, "num_steps":num_steps}

In [12]:
def count_words(text):
        """
        Count the number of words in the given text.
        
        Args:
            text (str): The input text to count words from.
        
        Returns:
            int: The number of words in the text.
        """
        # Split the text into words and count them
        words = text.split()
        return len(words)

def writing_node(state):
    """take the initial prompt and write a plan to make a long doc"""
    print("---WRITING THE DOC---")
    initial_instruction = state['initial_prompt']
    plan = state['plan']
    num_steps = int(state['num_steps'])
    num_steps += 1

    plan = plan.strip().replace('\n\n', '\n')
    planning_steps = plan.split('\n')
    text = ""
    responses = []
    if len(planning_steps) > 50:
        print("plan is too long")
        # print(plan)
        return
    for idx,step in enumerate(planning_steps):
        # Invoke the write_chain
        result = write_chain.invoke({
            "intructions": initial_instruction,
            "plan": plan,
            "text": text,
            "STEP": step
        })
        # result = step
        # print(f"----------------------------{idx}----------------------------")
        # print(step)
        # print("----------------------------\n\n")
        responses.append(result)
        text += result + '\n\n'

    final_doc = '\n\n'.join(responses)

    # Count words in the final document
    word_count = count_words(final_doc)
    print(f"Total word count: {word_count}")

    return {"final_doc": final_doc, "word_count": word_count, "num_steps":num_steps}

In [13]:
def saving_node(state):
    """take the finished long doc and save it to local disk as a .md file   """
    print("---SAVING THE DOC---")
    initial_prompt = state['initial_prompt']
    plan = state['plan']
    final_doc = state['final_doc']
    word_count = state['word_count']
    num_steps = int(state['num_steps'])
    num_steps += 1

    final_doc = final_doc + f"\n\nTotal word count: {word_count}"

    # save to local disk
    write_markdown_file(final_doc, f"final_doc")
    write_markdown_file(plan, f"plan")

    return { "num_steps":num_steps}

In [14]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_prompt: initial prompt
        plan: plan
        num_steps: number of steps
        llm_name: name of the LLM
        word_count: word count of the final document
    """
    initial_prompt : str
    plan : str
    num_steps : int
    final_doc : str
    write_steps : List[str]
    word_count : int

In [15]:
def create_workflow(llm):
    workflow = StateGraph(GraphState)
    workflow.add_node("planning_node", planning_node)
    workflow.add_node("writing_node", writing_node)
    workflow.add_node("saving_node", saving_node)
    workflow.set_entry_point("planning_node")
    workflow.add_edge("planning_node", "writing_node")
    workflow.add_edge("writing_node", "saving_node")
    workflow.add_edge("saving_node", END)
    return workflow.compile()

In [19]:
test_instruction = """
Please write a 1000 PRD on building a minimalistic AI powered note takking system that uses internet search
for context and tool to be used with an LLM. It also has a feature to quiz the user about the content
it generates so the user can remember the content better.
"""

inputs = {"initial_prompt": test_instruction,  
          "num_steps": 0,}

In [20]:
app = create_workflow(llm)

In [21]:
output = app.invoke(inputs)

---PLANNING THE WRITING---
<think>
Alright, I need to help break down a PRD into multiple subtasks for writing a 1000-word essay on building a minimalistic AI-powered note-taking system with quiz features and internet search integration. The user provided a detailed structure of the instruction and wants each paragraph broken down clearly.

First, understanding what each part of the instruction entails is crucial. The main points are: minimalism in design, integrating an LLM for context and tools, and adding a quiz feature to aid memory retention. Each subtask should cover one of these aspects.

Starting with the introduction, it's important to set the stage by defining the system and its purpose. This sets the foundation, so I'll allocate 200-300 words here. Next, explaining how the AI works ties into the LLM integration, which is a key component. This section should delve into search and tool suggestions.

The third subtask focuses on enhancing user interaction with quizzes. It's ess

KeyboardInterrupt: 